在计算机科学中，许多程序是为使一些问题得到最优解而写；例如，找到两点间的最短路径，找到最匹配一组点的线，或找到满足某些条件的最小对象集。计算机学家有许多策略来解决这些问题。这本书的目的之一就是为你揭示几个不同的解题策略。动态规划是这类求最优解问题的解决策略之一。

优化问题的一个典型例子就是用最少的硬币来找零。假设你是一家自动售货机制造商的程序员。你的公司正设法在每一笔交易找零时都能提供最少数目的硬币以便工作能更加简单。假设一个顾客投了1美元来购买37美分的物品。你用来找零的硬币的最小数量是多少？答案是六枚硬币：两个25 美分，一个10美分，三个1美分。我们是怎么得到六个硬币这个答案的呢？首先我们要使用面值中最大的硬币(25美分)，并且尽可能多的使用它，接着我们再使用下一个可使用的
最大面值的硬币，也是尽可能多的使用。这种方法被称为贪心算法，因为我们试图尽可能快的解决一个问题。


当我们使用美国硬币时，贪心算法工作的很好，但假设你的公司决定在Lower Elbonia（注释：漫画中杜撰的原东欧共产主义国家的南部）也部署自动售货机，那个地方除了有 1,5,10和25美分的硬币外，还有21美分的硬币。在这种情况下，贪心算法就不能找到63美分找零问题的最优解了。多了21面值的美分，贪心算法的答案仍是六个硬币，然而问题的最优解是三个21美分的硬币。

让我们来看看一个肯定能让我们找到问题的最优解的算法。既然这一章是关于递归的，你可能已经猜到我们将使用递归的方法解决问题。首先我们要弄清楚基本结束条件。如果我们要找的零钱的价值和某一种硬币的价值一样，那么答案很简单，只要一个硬币。

如果价值不匹配，我们就有几种选择。我们需要的是一个 1 美分加上给原始价值减去 1 美分找零所需硬币数量的最小值，或者一个 5 美分加上给原始价值减去 5 美分找零所需硬币数量的最小值，或者一个 10 美分加上给原始价值减去 10 美分找零所需硬币数量的最小值，等等。所以，给原始总数找零的硬币数量可以根据下面的方法计算:
\begin{split}   numCoins = min
\begin{cases}
1 + numCoins(original amount - 1) \\
1 + numCoins(original amount - 5) \\
1 + numCoins(original amount - 10) \\
1 + numCoins(original amount - 25)
\end{cases}
\label{eqn_change}\end{split}
下面代码中展示了我们刚才所描述的算法。在第3行，我们检查基本结束条件；也就是说，需要兑换的找零数等于我们硬币的某个面值。如果我们没有等于找零数目的硬币面额，那么我们就对每个小于我们找零总数的不同的硬币值调用递归。第6行展示了我们应该怎样通过使用一个硬币面值的列表，以帮助我们筛选出比当前找零价值小的硬币的列表。通过选定硬币的值，递归调用减小了我们需要找零的零钱总数。第7行展示了递归调用。注意，在同一行，我们要给硬币总数加1，这是因为我们使用了一枚硬币。只需加1就相当于：满足基本结束条件时，我们就做一次递归调用。

In [2]:
def recMC(coinValueList,change):
    minCoins = change
    if change in coinValueList:
        return 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recMC(coinValueList,change-i)
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins

print(recMC([1,5,10,25],63))

6


以上算法的问题就是它太低效了。事实上，它需要67716925次递归调用才能得出有4种硬币时找零63美分问题的最优解！为了理解我们的算法中的致命缺陷，观察图1.，它列出了我们为了找到兑换26美分的最优解时所需的377次函数调用中的一小部分。图中每一个节点对应一次recMC函数调用。节点上的数字显示了我们要计算的需要找零的硬币总量。箭头上的数字则显示了我们使用的硬币的面额。顺着图形，我们可以看到图中任何点的硬币的组合。主要问题就是我们做了大量的重复计算。例如，该图显示，这种算法会重复计算为15美分找零的最优解至少三次。每一次这种计算都要调用52次函数。显然我们浪费了大量的时间和精力来重复计算旧的结果。
![图1. 函数调用树](pic/4.12.1.png)
<center>图1. 函数调用树</center>

减少我们的工作量的关键在于记住一些出现过的结果，这样就能避免重复计算我们已经知道的结果。一个简单的解决方案就是我们将所找到的给硬币找零的最小数目存储在一个表中。然后在我们计算一个新的最小值之前，可以先查表看这个结果是否已知。如果表中已经有了这个结果，我们就可以从表中引用这个值而不是重复计算。有效编码1展示了包含了查表法的改善算法。

In [3]:
def recDC(coinValueList,change,knownResults):
    minCoins = change
    if change in coinValueList:
        knownResults[change] = 1
        return 1
    elif knownResults[change] > 0:
        return knownResults[change]
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recDC(coinValueList, change-i, knownResults)
            if numCoins < minCoins:
                minCoins = numCoins
                knownResults[change] = minCoins
    return minCoins

print(recDC([1,5,10,25],63,[0]*64))

6


注意，在第六行我们添加了一个测试来检查表中是否包含了为某个特定数目找零的硬币的最小值。如果没有，我们就调用递归来计算这个最小值并把它存储在表中。使用这个改进后的算法减少了我们得到了使用4种硬币找零63美分的问题答案，只需要221次递归函数调用！虽然上面代码的算法是正确的，但看起来感觉好像被黑客攻击过一样。此外，如果我们观察knowResults列表我们会发现，表中还有不少空洞。事实上，我们目前所采用的方法还不是动态规划，我们只是使用了一种叫做“函数值缓存”，或者一般称为“缓存”的方法改善了程序的性能。

真正的动态规划会采用更系统化的方法来解决问题。动态规划的解决方法是从为1分钱找零的最优解开始，逐步递加上去，直到我们需要的找零钱数。这就保证了在算法的每一步过程中，我们已经知道了兑换任何更小数值的零钱时所需的硬币数量的最小值。让我们来看看我们如何在11美分找零时用最小数目的硬币使用量来填表。图2.列出了过程。我们先从1美分开始。唯一可行的解决方案就是一个硬币(1美分)。下一行显示了为1美分和2美分找零的最小硬币数量。同样，唯一的答案是两个1美分硬币。在第5行，事情开始变得有趣了。现在我们考虑两种情况，五个1美分硬币或一个5美分硬币。我们如何决定那个才是做好的
选择呢？通过查表我们可以看到，为4美分找零的数量是四个，再加一个1美分变成了5美分，相当于有五个硬币。或者我们可以考虑0个硬币加上一个5美分硬币等于5美分，只有一个硬币。由于1比5小，我们把1存储在列表中。很快到了表的结尾，该考虑11美分了。图3. 显示了我们必须考虑的三种情况:
1. 一个1美分加上为 11-1=10美分找零的最小值(1)
2. 一个5美分加上为 11-5=6美分找零的最小值(2)
3. 一个10美分加上为 11-10=1美分找零的最小值(1)

情况1和3都给出了为11美分找零的最小值是2个硬币的答案。
![图2. 需要找零的硬币数量的最小值](pic/4.12.2.png)
<center>图2. 需要找零的硬币数量的最小值</center>
![图3. 考虑为11美分找零的最小值的3种方案](pic/4.12.3.png)
<center>图3. 考虑为11美分找零的最小值的3种方案</center>

下面代码是一个为解决找零问题所设计的动态规划算法。dpMakeChnge有三个参数：一个有效硬币面值的列表、我们想要兑换硬币的数值、一个包含所有部分找零最优解的列表。当函数运行完，minCoins会包含从0到所需兑换数值的每一个数值对应的最优解。

In [7]:
def dpMakeChange(coinValueList,change,minCoins):
    for cents in range(change+1):
        coinCount = cents
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
        minCoins[cents] = coinCount
    return minCoins[change]

注意，dpMakeChange不是一个递归函数，即使我们开始使用了递归解决这个问题。必须要认识到的是，你可以写一个递归算法来解决问题，但这并不意味着它就是最好和最有效的解决方案。这个函数的大部分内容是做循环，循环从第4行开始。在这个循环中，我们要考虑使用所有可能的硬币面值来为cents中所指定的数值兑换硬币。像我们在上面举的给11分钱兑换硬币的例子，我们把部分找零的最优解记录下来并保存在minCoins列表中。

尽管我们的找零算法在找出所需硬币数量的最小值上做得很好，但是它并不能真的帮助我们兑换硬币，因为我们没有跟踪记录我们使用的硬币。我们可以很容易地扩展dpMakeChange来跟踪记录我们所使用的硬币，只要简单的记录我们为minCoins的每一项添加的最后一个硬币就可以了。如果我们知道了最后一个添加的硬币，就可以简单的减去这个硬币的币值来找到最优解列表中之前的一项进行找零。我们可以一直倒退访问列表直到回到列表的最开始。

下面的代码显示了改善了性能的dpMakeChnge算法，它能够跟踪硬币使用的路径，同时有一个printCoins的功能，通过重访列表，打印出每个使用过的硬币的值。这表明了这个算法可以解决我们在Lower Elbonia的朋友的问题。main的前两行设置了要转换的总量，并且创建了可使用的硬币面值的列表。接下来的两行创建了我们存储结果的列表。coinUsed 是一个我们用来找零的硬币的列表，coinCount是为表中相应的位置的量找零所需的硬币数量的最小值。

注意我们打印出的硬币值直接来自coinUsed阵列。第一次调用函数时，我们从阵列的63位置开始并打印出21。然后我们得到63-21=42并观察列表的第42级。我们又一次发现存储了21。最后，列表的21级也包含了21，最后我们得到了三个21分硬币。

In [6]:
def dpMakeChange(coinValueList,change,minCoins,coinsUsed):
    for cents in range(change+1):
        coinCount = cents
        newCoin = 1
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
                newCoin = j
        minCoins[cents] = coinCount
        coinsUsed[cents] = newCoin
    return minCoins[change]

def printCoins(coinsUsed,change):
    coin = change
    while coin > 0:
        thisCoin = coinsUsed[coin]
        print(thisCoin)
        coin = coin - thisCoin

def main():
    amnt = 63
    clist = [1,5,10,21,25]
    coinsUsed = [0]*(amnt+1)
    coinCount = [0]*(amnt+1)

    print("Making change for",amnt,"requires")
    print(dpMakeChange(clist,amnt,coinCount,coinsUsed),"coins")
    print("They are:")
    printCoins(coinsUsed,amnt)
    print("The used list is as follows:")
    print(coinsUsed)

main()

Making change for 63 requires
3 coins
They are:
21
21
21
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21]
